In [ ]:
import pandas as pd, matplotlib.pyplot as plt, re, os, itertools, collections, tqdm, textwrap, numpy as np
from matplotlib import rcParams
rcParams['font.family'] = "P052"
%config InlineBackend.figure_format = 'svg'

In [ ]:
# See how many sites PSP and atlas have in common from the files /Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/site_list_86201.txt and /Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/PSP_site_list.csv

# Read in the site list from the atlas
site_list_atlas = pd.read_csv('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/site_list_86201.txt', header=None)
site_list_psp = pd.read_csv('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/PSP_site_list.csv', header=None)
# Create a Venn diagram from the two lists
from matplotlib_venn import venn2
venn2([set(site_list_atlas[0]), set(site_list_psp[0])], set_labels = ('Atlas', 'PSP'))
plt.show()

##### Since there is no overlap, we have to scrape from the Nature Atlas

In [ ]:
### Assume we have already run scrape.py

# Open all files in scraped directory and create a mapping from site name to results table
scraped_path = '/Users/druc594/Downloads/KinLibDown'
mapping = {}
for file in tqdm.tqdm([x for x in os.listdir(scraped_path) if x.endswith('.tsv')]):
    site_name = ""
    tab = None
    try:
        tab = pd.read_csv(os.path.join(scraped_path, file), sep='\t')
        site_name = re.sub(r'-[0-9]+`[0-9]+\.tsv', '', file)
    except Exception as e:
        print(e)
        print("Problem:", file)
    
    mapping[site_name] = tab
        
for k, v in mapping.items():
    mapping[k] = v.set_index("kinase").to_dict()['site_percentile']
THEIR_RESULTS_FILE = "./41586_2022_5575_MOESM5_ESM.csv"
their_results = pd.read_csv(THEIR_RESULTS_FILE).set_index("Uniprot Primary Accession")
matrix_name_to_uniprot_id: dict[str, str] = (
    pd.read_csv("./41586_2022_5575_MOESM3_ESM.csv").set_index("Matrix_name").to_dict()["Uniprot id"]
)

uniprot_id_to_matrix_name = {v: k for k, v in matrix_name_to_uniprot_id.items()}
assert len(uniprot_id_to_matrix_name) == len(matrix_name_to_uniprot_id)
kin_uniprot_to_sites = collections.defaultdict(list[str])
psp = pd.read_excel("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/data/raw_data/PSP_script_download.xlsx")[["SITE_+/-7_AA", "KIN_ACC_ID"]]

for _, row in psp.iterrows():
    assert isinstance(row["SITE_+/-7_AA"], str)
    kin_uniprot_to_sites[row["KIN_ACC_ID"]].append(re.sub(r"^(.{8})", r"\1*", row["SITE_+/-7_AA"].upper()))
mod_cols = set([matrix_name_to_uniprot_id[re.sub(r"^([0-9A-Z]+)_rank.*", r"\1", x)] for x in their_results.columns if x.endswith("_rank")])
kuts = kin_uniprot_to_sites.copy().keys()

for kin_u in kuts:
    if kin_u not in mod_cols:
        kin_uniprot_to_sites.pop(kin_u)
site_to_kin_uniprots = collections.defaultdict(list[str])

for kin_u, sites in kin_uniprot_to_sites.items():
    for site in sites:
        site_to_kin_uniprots[site].append(kin_u)

site_to_kin_uniprots_shortened = {k.replace("*", "")[2:-1].replace("_", ""):v for k,v in site_to_kin_uniprots.items()}
site_to_percentiles = collections.defaultdict(list[float])

# Build up a mapping from site to list of percentiles
for site in mapping:
    assert site in site_to_kin_uniprots_shortened, f"{site} not in site_to_kin_uniprots_shortened"
    for kin in site_to_kin_uniprots_shortened[site]:
        assert kin in uniprot_id_to_matrix_name, f"{kin} not in uniprot_id_to_matrix_name"
        assert site in mapping, f"{site} not in mapping"
        if uniprot_id_to_matrix_name[kin] in mapping[site]:
            site_to_percentiles[site].append(mapping[site][uniprot_id_to_matrix_name[kin]])
            
all_percentiles = list(itertools.chain(*site_to_percentiles.values()))

#### Make a histogram of the percentiles

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
max_height = 0
hist_data = ax.hist(all_percentiles, bins=list(range(0, 101, 5)))
this_max_height = max(hist_data[0])
assert isinstance(this_max_height, float)
max_height = max(max_height, this_max_height)
max_height = np.ceil(max_height / 5) * 5
ax.set_title(
    textwrap.fill(
        (
            f"Distribution of Atlas-predicted percentiles for {len(all_percentiles)} experimentally validated PSP K-S"
            " pairs"
        ),
        120,
    )
)
ax.set_xlabel("Percentile")
ax.set_ylabel("Count")
ax.set_xticks([1], [f"{len(all_percentiles)} pairs"])
ax.set_xticks(range(0, 101, 10), range(0, 101, 10))
ax.set_ylim(0, min(4 + max_height, max_height * 1.1))
print(max_height)
ytk = range(0, int(max_height) + 1, int(5*np.floor(np.log10(max_height))))
ax.set_yticks(ytk, ytk)
_ = ax.grid(visible=True, axis="y", linestyle="--", alpha=0.4)

#### Make a boxplot of the percentiles

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (3, 7))
max_height = 0
ax.boxplot(all_percentiles)
# this_max_height = max(hist_data[0])
# assert isinstance(this_max_height, float)
# max_height = max(max_height, this_max_height)
# max_height = np.ceil(max_height/5) * 5
ax.set_title(textwrap.fill(f"Distribution of Atlas-predicted percentiles for {len(all_percentiles)} experimentally validated PSP K-S pairs", 40))
# ax.set_xlabel("Percentile")
ax.set_ylabel("Percentile")
ax.set_xticks([1], [f"{len(all_percentiles)} pairs"])
# ax.set_xticks(range(0, 101, 10), range(0, 101, 10))
# ax.set_ylim(0, min(4 + max_height, max_height*1.1))
# print(max_height)
_ = ax.set_yticks(range(0, 101, 10))
ax.grid(visible=True, axis = 'y', linestyle="--", alpha=0.4)